In [ ]:
# ===============================
# Deep Neural Network from Scratch using Optuna for Hyperparameter Optimization
# Dataset: Iris (Multiclass Classification)
# ===============================

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from functools import partial
import optuna


# =======================================
# 1. Load and Preprocess Data
# =======================================
iris = datasets.load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# Normalize features for better convergence
scaler = RobustScaler()
X = scaler.fit_transform(X)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

def train_val_test_split(X, y, train_size=0.7, val_size=0.15, test_size=0.15, random_state=42):
    assert abs(train_size + val_size + test_size - 1.0) < 1e-6, "Sizes must sum to 1"

    # First split: train vs temp (val+test)
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=(1 - train_size), random_state=random_state, stratify=y)

    # Compute proportion of validation relative to temp
    val_prop = val_size / (val_size + test_size)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(1 - val_prop), random_state=random_state, stratify=y_temp)

    return X_train, X_val, X_test, y_train, y_val, y_test

# Usage:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y)
print(X_train.shape, X_val.shape, X_test.shape)


# =======================================
# 2. Activations
# =======================================
def sigmoid(x, is_derivative=False):
    if is_derivative:
        return np.exp(-x) / ((1 + np.exp(-x)) ** 2)
    return 1 / (1 + np.exp(-x))

def tanh(x, is_derivative=False):
    if is_derivative:
        return 1 - np.tanh(x) ** 2
    return np.tanh(x)

def relu(x, is_derivative=False):
    if is_derivative:
        return (x > 0).astype(float)
    return np.maximum(0, x)

def leaky_relu(x, is_derivative=False, alpha=0.01):
    if is_derivative:
        x = np.where(x < 0, alpha, 1)
        return x
    return np.maximum(alpha * x, x)

def softmax(x):
    exp_shifted = np.exp(x - np.max(x, axis=0, keepdims=True))
    return exp_shifted / np.sum(exp_shifted, axis=0, keepdims=True)


# =======================================
# 3. Loss Functions
# =======================================
def mse_loss(Y, Y_pred):
    m = Y.shape[0]
    return np.sum((Y - Y_pred.T) ** 2) / (2 * m)

def cross_entropy_loss(Y, Y_pred, eps=1e-9):
    m = Y.shape[0]
    return -np.sum(Y * np.log(Y_pred.T + eps)) / m


# =======================================
# 4. Initialization
# =======================================
def initialize_parameters(layer_dims):
    np.random.seed(42)
    params = {}
    for i in range(1, len(layer_dims)):
        params[f"W{i}"] = np.random.randn(layer_dims[i], layer_dims[i - 1]) * np.sqrt(1. / layer_dims[i - 1])
        params[f"b{i}"] = np.zeros((layer_dims[i], 1))
    return params


# =======================================
# 5. Forward
# =======================================
def forward_propagation(X, params, activations, dropout_rates=None, training=True):
    cache = {"A0": X.T}
    L = len(activations)
    dropout_masks = {}

    for i in range(1, L + 1):
        W, b = params[f"W{i}"], params[f"b{i}"]
        A_prev = cache[f"A{i-1}"]
        Z = np.dot(W, A_prev) + b

        if activations[i-1] == "sigmoid": A = sigmoid(Z)
        elif activations[i-1] == "relu": A = relu(Z)
        elif activations[i-1] == "tanh": A = tanh(Z)
        elif activations[i-1] == "leaky_relu": A = leaky_relu(Z)
        elif activations[i-1] == "softmax": A = softmax(Z)
        else: raise ValueError(f"Unsupported activation: {activations[i-1]}")

        # Dropout
        if training and dropout_rates and i <= len(dropout_rates):
            rate = dropout_rates[i - 1]
            mask = (np.random.rand(*A.shape) > rate).astype(float)
            A *= mask
            A /= (1 - rate)
            dropout_masks[f"M{i}"] = mask

        cache[f"Z{i}"], cache[f"A{i}"] = Z, A

    return cache[f"A{L}"], cache, dropout_masks


# =======================================
# 6. Backpropagation
# =======================================
def backward_propagation(Y, params, cache, activations, dropout_masks=None, dropout_rates=None, weight_decay=0.0):
    grads = {}
    m = Y.shape[0]
    L = len(activations)
    Y = Y.T
    dZ = cache[f"A{L}"] - Y

    for i in reversed(range(1, L + 1)):
        A_prev = cache[f"A{i-1}"]
        W = params[f"W{i}"]

        grads[f"dW{i}"] = (1 / m) * np.dot(dZ, A_prev.T) + weight_decay * W
        grads[f"db{i}"] = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
        grads = {k: np.clip(v, -1, 1) for k, v in grads.items()}

        if i > 1:
            Z_prev = cache[f"Z{i-1}"]
            dA_prev = np.dot(W.T, dZ)

            if dropout_masks and i - 1 in [int(k[1:]) for k in dropout_masks.keys()]:
                mask = dropout_masks[f"M{i-1}"]
                dA_prev *= mask
                dA_prev /= (1 - dropout_rates[i - 2])

            if activations[i - 2] == "sigmoid":
                dZ = dA_prev * sigmoid(Z_prev, is_derivative=True)
            elif activations[i - 2] == "relu":
                dZ = dA_prev * relu(Z_prev, is_derivative=True)
            elif activations[i - 2] == "tanh":
                dZ = dA_prev * tanh(Z_prev, is_derivative=True)
            elif activations[i - 2] == "leaky_relu":
                dZ = dA_prev * leaky_relu(Z_prev, is_derivative=True)

    return grads


# =======================================
# 7. Optimizers
# =======================================
'''
    Stochatic Gradient Descent (SGD):
    θ^(t+1) <- θ^t - η∇L(y, ŷ)

    Momentum:
    v^(t+1) <- βv^t + (1-β)∇L(y, ŷ)^t
    θ^(t+1) <- θ^t - ηv^(t+1)
'''
def update_parameters(params, grads, lr, optimizer_type, velocity=None, beta=0.9):
    if optimizer_type == "sgd":
        for i in range(1, len(params)//2 + 1):
            params[f"W{i}"] -= lr * grads[f"dW{i}"]
            params[f"b{i}"] -= lr * grads[f"db{i}"]

    elif optimizer_type == "momentum":
        if velocity is None:
            velocity = {k: np.zeros_like(v) for k, v in params.items()}

        for i in range(1, len(params)//2 + 1):
            velocity[f"W{i}"] = beta * velocity[f"W{i}"] + (1 - beta) * grads[f"dW{i}"]
            velocity[f"b{i}"] = beta * velocity[f"b{i}"] + (1 - beta) * grads[f"db{i}"]
            params[f"W{i}"] -= lr * velocity[f"W{i}"]
            params[f"b{i}"] -= lr * velocity[f"b{i}"]

    return params, velocity


# =======================================
# 8. Training Loop
# =======================================
def train_neural_network(
    X_train, 
    Y_train,
    X_val, 
    Y_val,
    hidden_layers=[10], 
    lr=0.01, 
    epochs=500,
    batch_size=16,
    activations=None, 
    loss_function="cross_entropy",
    optimizer_type="momentum",
    dropout_rates=[0.1],
    weight_decay=1e-4, 
    early_stopping=True, 
    patience=100,
    print_every=100
):
    input_dim, output_dim = X_train.shape[1], Y_train.shape[1]
    layer_dims = [input_dim] + hidden_layers + [output_dim]
    if activations is None:
        activations = ["relu"] * len(hidden_layers) + ["softmax"]

    params = initialize_parameters(layer_dims)
    velocity = None
    best_loss = np.inf
    patience_counter = 0

    for epoch in range(epochs):
        # Shuffle
        perm = np.random.permutation(X_train.shape[0])
        X_shuffled, Y_shuffled = X_train[perm], Y_train[perm]

        # Mini-batch iteration
        if batch_size is None:
            batches = [(X_shuffled, Y_shuffled)]
        else:
            batches = [(X_shuffled[i:i+batch_size], Y_shuffled[i:i+batch_size]) for i in range(0, X_shuffled.shape[0], batch_size)]

        # Training by batch
        epoch_loss = 0.0
        for X_batch, Y_batch in batches:
            A_out, cache, dropout_masks = forward_propagation(X_batch, params, activations, dropout_rates)
            loss = cross_entropy_loss(Y_batch, A_out)
            grads = backward_propagation(Y_batch, params, cache, activations, dropout_masks, dropout_rates, weight_decay)
            params, velocity = update_parameters(params, grads, lr, optimizer_type, velocity)
            epoch_loss += loss

        # Validation loss
        A_val, _, _ = forward_propagation(X_val, params, activations, training=False)
        val_loss = cross_entropy_loss(Y_val, A_val) if loss_function == "cross_entropy" else mse_loss(Y_val, A_val)
        loss = epoch_loss / len(batches)

        if epoch % print_every == 0:
            print(f"Epoch {epoch+1:3d}/{epochs} | Train Loss: {loss:.4f} | Val Loss: {val_loss:.4f}")


        # Early stopping
        if early_stopping:
            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
                best_params = params.copy()
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping triggered.")
                    params = best_params
                    break

    return params


# =======================================
# 9. Prediction & Evaluation
# =======================================
def predict(X, params, activations):
    A_out, _, _ = forward_propagation(X, params, activations, training=False)
    return np.argmax(A_out, axis=0)

def accuracy_score(y_true, y_pred):
    return np.mean(y_true == y_pred) * 100


# =======================================
# 10. Hyperparameter Optimization with Optuna
# =======================================
def objective(trial, loss_function="cross_entropy", epochs=1000):
    # Hyperparameters to search
    n_layers = trial.suggest_int("n_layers", 1, 3)
    hidden_layers = [trial.suggest_int(f"n_units_l{i}", 5, 128) for i in range(n_layers)]
    lr = trial.suggest_float("lr", 1e-4, 0.1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    dropout_rates = [trial.suggest_float(f"dropout_l{i}", 0.0, 0.5) for i in range(n_layers)]
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    optimizer_type = trial.suggest_categorical("optimizer", ["sgd", "momentum"])
    stopping_patience = trial.suggest_int("stopping_patience", 50, 500)
    # Activation functions per layer
    possible_activations = ["relu", "sigmoid", "tanh", "leaky_relu"]
    activations = [trial.suggest_categorical(f"activation_l{i}", possible_activations) for i in range(n_layers)]
    activations.append("softmax")  # output layer always softmax

    # Train network
    params = train_neural_network(
        X_train, 
        y_train,
        X_val, 
        y_val,
        hidden_layers=hidden_layers,
        lr=lr,
        epochs=epochs,
        batch_size=batch_size,
        activations=activations,
        loss_function=loss_function,
        optimizer_type=optimizer_type,
        dropout_rates=dropout_rates,
        weight_decay=weight_decay,
        early_stopping=True,
        patience=stopping_patience,
        print_every=100
    )

    # Optuna tries to minimize the loss function
    A_val, _, _ = forward_propagation(X_val, params, activations, training=False)
    val_loss = cross_entropy_loss(y_val, A_val) if loss_function == "cross_entropy" else mse_loss(y_val, A_val)
    return val_loss

# Run Optuna study
loss_function = "cross_entropy"
epochs = 1000
obj = partial(objective, loss_function=loss_function, epochs=epochs)
study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(obj, n_trials=30)
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value:.4f}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


# =======================================
# 11. Retrain Best Model on Train + Val and Evaluate on Test
# =======================================
best_params = trial.params
n_layers = best_params["n_layers"]
hidden_layers = [best_params[f"n_units_l{i}"] for i in range(n_layers)]
lr = best_params["lr"]
batch_size = best_params["batch_size"]
dropout_rates = [best_params[f"dropout_l{i}"] for i in range(n_layers)]
weight_decay = best_params["weight_decay"]
optimizer_type = best_params["optimizer"]
stopping_patience = best_params["stopping_patience"]
activations = [best_params[f"activation_l{i}"] for i in range(n_layers)]
activations.append("softmax")  # output layer always softmax

# Combine train and val sets
final_params = train_neural_network(
    X_train, 
    y_train,
    X_val, 
    y_val,
    hidden_layers=hidden_layers,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
    activations=activations,
    loss_function=loss_function,
    optimizer_type=optimizer_type,
    dropout_rates=dropout_rates,
    weight_decay=weight_decay,
    early_stopping=True,
    patience=stopping_patience,
    print_every=100
)

# Evaluate on test set
y_test_pred = predict(X_test, final_params, activations)
y_test_true = np.argmax(y_test, axis=1)
test_accuracy = accuracy_score(y_test_true, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}%")

[I 2025-10-25 15:36:13,537] A new study created in memory with name: no-name-ae0d4c8c-5584-451c-80bf-16526d0dd98c


(104, 4) (23, 4) (23, 4)
Epoch   1/1000 | Train Loss: 1.0897 | Val Loss: 1.0276
Epoch 101/1000 | Train Loss: 0.6610 | Val Loss: 0.6826
Epoch 201/1000 | Train Loss: 0.5436 | Val Loss: 0.5623
Epoch 301/1000 | Train Loss: 0.4724 | Val Loss: 0.5040
Epoch 401/1000 | Train Loss: 0.4478 | Val Loss: 0.4691
Epoch 501/1000 | Train Loss: 0.4102 | Val Loss: 0.4453
Epoch 601/1000 | Train Loss: 0.4061 | Val Loss: 0.4272
Epoch 701/1000 | Train Loss: 0.3693 | Val Loss: 0.4126


[I 2025-10-25 15:36:14,120] Trial 0 finished with value: 0.37874910825272123 and parameters: {'n_layers': 1, 'n_units_l0': 45, 'lr': 0.0005207830247539784, 'batch_size': 16, 'dropout_l0': 0.2314208190583974, 'weight_decay': 6.0034171424595095e-05, 'optimizer': 'momentum', 'stopping_patience': 436, 'activation_l0': 'tanh'}. Best is trial 0 with value: 0.37874910825272123.


Epoch 801/1000 | Train Loss: 0.3501 | Val Loss: 0.3998
Epoch 901/1000 | Train Loss: 0.3374 | Val Loss: 0.3887
Epoch   1/1000 | Train Loss: 1.0815 | Val Loss: 1.0435
Epoch 101/1000 | Train Loss: 0.5841 | Val Loss: 0.5791
Epoch 201/1000 | Train Loss: 0.4161 | Val Loss: 0.4617
Epoch 301/1000 | Train Loss: 0.3168 | Val Loss: 0.4042
Epoch 401/1000 | Train Loss: 0.2781 | Val Loss: 0.3651
Epoch 501/1000 | Train Loss: 0.2512 | Val Loss: 0.3340
Epoch 601/1000 | Train Loss: 0.2302 | Val Loss: 0.3075
Epoch 701/1000 | Train Loss: 0.2127 | Val Loss: 0.2845
Epoch 801/1000 | Train Loss: 0.2016 | Val Loss: 0.2642
Epoch 901/1000 | Train Loss: 0.1792 | Val Loss: 0.2476


[I 2025-10-25 15:36:14,804] Trial 1 finished with value: 0.2331473661518831 and parameters: {'n_layers': 2, 'n_units_l0': 21, 'n_units_l1': 124, 'lr': 0.0018926228140705523, 'batch_size': 32, 'dropout_l0': 0.008331749270429845, 'dropout_l1': 0.07679814715820144, 'weight_decay': 5.028146803675402e-06, 'optimizer': 'momentum', 'stopping_patience': 82, 'activation_l0': 'tanh', 'activation_l1': 'leaky_relu'}. Best is trial 1 with value: 0.2331473661518831.


...

Epoch 901/1000 | Train Loss: 0.0903 | Val Loss: 0.0699
Epoch   1/1000 | Train Loss: 1.1086 | Val Loss: 0.9710
Epoch 101/1000 | Train Loss: 0.3844 | Val Loss: 0.1770
Epoch 201/1000 | Train Loss: 0.3321 | Val Loss: 0.1495
Epoch 301/1000 | Train Loss: 0.3022 | Val Loss: 0.1498
Epoch 401/1000 | Train Loss: 0.2096 | Val Loss: 0.0835
Epoch 501/1000 | Train Loss: 0.2059 | Val Loss: 0.0781
Epoch 601/1000 | Train Loss: 0.1733 | Val Loss: 0.0774
Epoch 701/1000 | Train Loss: 0.2235 | Val Loss: 0.0618
Epoch 801/1000 | Train Loss: 0.1819 | Val Loss: 0.0609
Epoch 901/1000 | Train Loss: 0.1483 | Val Loss: 0.0580


[I 2025-10-25 15:37:05,201] Trial 28 finished with value: 0.049998391487115434 and parameters: {'n_layers': 3, 'n_units_l0': 75, 'n_units_l1': 90, 'n_units_l2': 114, 'lr': 0.028215713227873487, 'batch_size': 8, 'dropout_l0': 0.32370263488861745, 'dropout_l1': 0.36154424711516175, 'dropout_l2': 0.3142262371342558, 'weight_decay': 0.0027060904393533614, 'optimizer': 'sgd', 'stopping_patience': 378, 'activation_l0': 'sigmoid', 'activation_l1': 'tanh', 'activation_l2': 'leaky_relu'}. Best is trial 10 with value: 0.0008098299657081411.


Epoch   1/1000 | Train Loss: 1.0768 | Val Loss: 0.7698
Epoch 101/1000 | Train Loss: 0.0869 | Val Loss: 0.1478
Epoch 201/1000 | Train Loss: 0.0702 | Val Loss: 0.1308
Epoch 301/1000 | Train Loss: 0.0710 | Val Loss: 0.1283
Epoch 401/1000 | Train Loss: 0.0632 | Val Loss: 0.1171
Epoch 501/1000 | Train Loss: 0.0535 | Val Loss: 0.1146
Epoch 601/1000 | Train Loss: 0.0595 | Val Loss: 0.1079
Epoch 701/1000 | Train Loss: 0.0619 | Val Loss: 0.1068
Epoch 801/1000 | Train Loss: 0.0484 | Val Loss: 0.1033


[I 2025-10-25 15:37:05,896] Trial 29 finished with value: 0.10132341998757717 and parameters: {'n_layers': 1, 'n_units_l0': 106, 'lr': 0.05619399082307371, 'batch_size': 16, 'dropout_l0': 0.18854567921891188, 'weight_decay': 0.000767094921314456, 'optimizer': 'momentum', 'stopping_patience': 252, 'activation_l0': 'tanh'}. Best is trial 10 with value: 0.0008098299657081411.


Epoch 901/1000 | Train Loss: 0.0563 | Val Loss: 0.1007
Best trial:
  Value: 0.0008
  Params:
    n_layers: 3
    n_units_l0: 128
    n_units_l1: 125
    n_units_l2: 128
    lr: 0.08872966878731414
    batch_size: 8
    dropout_l0: 0.4719034296249297
    dropout_l1: 0.30899072246268755
    dropout_l2: 0.4933546876716215
    weight_decay: 5.1478572239525056e-05
    optimizer: sgd
    stopping_patience: 211
    activation_l0: leaky_relu
    activation_l1: tanh
    activation_l2: tanh
Epoch   1/1000 | Train Loss: 0.6496 | Val Loss: 0.4159
Epoch 101/1000 | Train Loss: 0.0746 | Val Loss: 0.0124
Epoch 201/1000 | Train Loss: 0.0598 | Val Loss: 0.0197
Epoch 301/1000 | Train Loss: 0.0113 | Val Loss: 0.0044
Epoch 401/1000 | Train Loss: 0.0380 | Val Loss: 0.0024
Epoch 501/1000 | Train Loss: 0.0196 | Val Loss: 0.0013
Epoch 601/1000 | Train Loss: 0.0070 | Val Loss: 0.0038
Epoch 701/1000 | Train Loss: 0.0079 | Val Loss: 0.0007
Epoch 801/1000 | Train Loss: 0.0032 | Val Loss: 0.0008
Epoch 901/1000 | Tr